<a href="https://colab.research.google.com/github/etherealxx/volatile-concentration-localux-colab/blob/main/volatile_concentration_localux_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***`VCL-colab`***
## All camenduru colab in one spot, synced in realtime

##  
To use:
- Click the play button
- Wait for less than a minute, until a UI with **big orange button** shows up
- Choose the model you want, then click the orange button
- Just wait for about 8 minute until the webui links shows up on the bottom

About the options:
- `repo_branch`: Read [here](https://github.com/camenduru/stable-diffusion-webui-colab) (just use `stable` as default)
- `choose_models`: Untick this if you're relaunching the UI, or if you don't want to choose model (The webui will always download sd 1.4 model instead)
- `controlnet_model`: Tick this if you want to download controlnet models (for controlnet usage). Untick this if you're relaunching the UI. Note that `lite` branch won't download any controlnet model regardless if this checkbox is being ticked or not.

#  
(open the hidden cell to show help☝️)

In [ ]:
#@title 👇 Run this
repo_branch = 'stable' #@param ["lite", "stable", "nightly"]
choose_models = True #@param {type:"boolean"}
controlnet_models = False #@param {type:"boolean"}
%cd /content
%env TF_CPP_MIN_LOG_LEVEL=1
!git clone https://github.com/etherealxx/volatile-concentration-localux-colab /content/colabtools
!pip install -q gradio

colaboptions = {
  "branch": repo_branch,
  "filename": "stable_diffusion_1_5_webui_colab.ipynb",
  "part": "part2",
  "controlnet": controlnet_models,
  "download_model": choose_models
} #default

import os, pickle
import gradio as gr
from IPython.display import clear_output

!python /content/colabtools/cloneall2.py
!python /content/colabtools/choosemodel3.py {repo_branch}

if os.path.exists('/content/sortedcolabname.pkl'):
  with open('/content/sortedcolabname.pkl', 'rb') as f:
      sortedcolabname = pickle.load(f)

clear_output(wait=True)

everycolab = f'/content/camendurus/{repo_branch}'
modelischosen = False
modelchosenmessage = ''
chosencolabname = ''

def modelchosen(chosenmodel):
  global modelischosen
  global modelchosenmessage
  global chosencolabname
  global colaboptions
  global choose_models
  modelischosen = True
  if chosenmodel:
    modelchosenmessage = "[1;32mModel " + chosenmodel + " was chosen. It'll be downloaded soon. Continue scroll down and wait for the webui to be loaded."
    textbox_text = f"Model '{chosenmodel} was chosen. It'll be downloaded soon. Continue scroll down and wait for the webui to be loaded."
    chosencolabname = chosenmodel
  else:
    modelchosenmessage = "[1;32mNo model was chosen. Continue scroll down and wait for the webui to be loaded."
    textbox_text = f"No model was chosen. Continue scroll down and wait for the webui to be loaded."
    choose_models = False
    colaboptions["download_model"] = False
  return [gr.Button.update(visible=False), gr.Textbox.update(value=textbox_text, visible=True)]

with gr.Blocks() as vclcolab:
    with gr.Column():
      choose = gr.Dropdown(sortedcolabname, label="Choose Your Model")
      confirm = gr.Button("Use This Model", variant="primary", visible=True)
      donetext = gr.Textbox("", label="", visible=False)
      confirm.click(modelchosen, inputs=[choose], outputs=[confirm, donetext])

if choose_models:
  vclcolab.launch(server_port=7859)

  while True:
    if modelischosen:
      clear_output(wait=True)
      print(modelchosenmessage)
      print('[0m')
      vclcolab.close()
      break

aria2c_lines = []
colabfilename = ''
chosencolabfilepath = os.path.join(everycolab, colaboptions["filename"])
                                   
if chosencolabname:
   colabfilename = chosencolabname + '_webui_colab.ipynb'
   colaboptions["filename"] = colabfilename
   chosencolabfilepath = os.path.join(everycolab, colabfilename)

if os.path.exists(chosencolabfilepath):
  print("[1;32mGathering aria2c code from " + chosencolabfilepath + "...")
  print('[0m')
  with open(chosencolabfilepath, 'r', encoding='utf-8') as f:
      for line in f:
          stripped_line = line.strip()
          if stripped_line.startswith('"!aria2c'):
              aria2c_lines.append(stripped_line)

if aria2c_lines:
  with open('/content/arialist.pkl', 'wb') as f:
      pickle.dump(aria2c_lines, f)

with open('/content/colaboptions.pkl', 'wb') as f:
    pickle.dump(colaboptions, f)

# !python /content/colabtools/camendurucolab.py {colabfilename} {repo_branch} part1
!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb
!python /content/colabtools/camendurucolab.py #{colabfilename} {repo_branch} part2
!python /content/colabtools/runaria2.py
%cd /content/volatile-concentration-localux
# !python /content/colabtools/camendurucolab.py {colabfilename} {repo_branch} part3
!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/volatile-concentration-localux/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/volatile-concentration-localux/launch.py
!sed -i -e 's/\"sd_model_checkpoint\"\,/\"sd_model_checkpoint\,sd_vae\,CLIP_stop_at_last_layers\"\,/g' /content/volatile-concentration-localux/modules/shared.py
print("[1;32mlaunching the webui...")
print('[0m')
!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple

(Doesn't work with Bromite browser on mobile. Use Chrome instead.)

#  

 ### (debug area)

In [ ]:
%cd /content
!git clone https://github.com/etherealxx/volatile-concentration-localux-colab /content/colabtools

In [ ]:
!rm -rf /content/volatile-concentration-localux
!rm -rf /content/camendurus
!rm /content/frpc_linux_amd64

In [ ]:
import os, pickle
if os.path.exists('/content/arialist.pkl'):
  with open('/content/arialist.pkl', 'rb') as f:
      arialines2 = pickle.load(f)

for line in arialines2:
  print(line)

In [ ]:
repo_branch = 'nightly' #@param ["lite", "stable", "nightly"]

colaboptions = {
  "branch": repo_branch,
  "filename": "stable_diffusion_1_5_webui_colab.ipynb",
  "part": "part2"
} #default


print(colaboptions['branch'])

In [ ]:
with open('/content/camendurus/nightly/a_certain_model_webui_colab.ipynb', 'r', encoding='utf-8') as f:
    for line in f:
        stripped_line = line.strip()
        if stripped_line.startswith('"!aria2c'):
            # aria2c_lines.append(stripped_line)
            print("[1;32m" + stripped_line)
            print('[0m')
        else:
          print(stripped_line)

In [ ]:
checktest = None
with open('/content/arialist.pkl', 'rb') as f:
  checktest = pickle.load(f)
print(checktest)